In [43]:
import urls
import os
import sys

class FileReader(object):
    def __init__(self, filePath):
        self.filePath = filePath
    
    def read(self):
        with open(self.filePath, 'r', encoding="utf8") as f:
            s = f.read()
        return s

    def read_lines(self):
        with open(self.filePath, 'r', encoding="utf8") as f:
            stopwords = set([w.strip() for w in f.readlines()])
            return stopwords
fr = FileReader(urls.STOP_WORDS).read_lines()
print("là" not in fr)

False


In [44]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import string
import re


class NLP(object):
    def __init__(self, text=None):
        self.text = text
        self.stopwords = FileReader(urls.STOP_WORDS).read_lines()
        print(str(len(self.stopwords)))
        self.newWords = []
        
    def remove_tags(self, raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html )
        return cleantext

    def remove_links(self, text):
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        return text

    def remove_punctuation(self, text):
        return text.translate(str.maketrans('', '', string.punctuation))
        

    def remove_stopwords(self, words, stopwords):
        important_words = []
        for index in range(len(words)):
            if (words[index] not in stopwords ):
                important_words.append(words[index])
        self.tokens = important_words
        return important_words
    
    def lower_sentences(self, text):
         return text.lower()

    def tokenize_sentences(self, text):
        return ViTokenizer.tokenize(text)

    def segment_word(self, tokens):
        return ViPosTagger.postagging(tokens)
    
    def keep_verb_noun(self, postags):
        res = []
        for index in range(len(postags[0])):
            if postags[1][index] == 'N' or postags[1][index] == 'V':
                res.append(postags[0][index])
        return res

    def remove_digits(self, tokens):
        new_words = []
        for index in range(len(tokens)):
            if not tokens[index].isdigit():
                new_words.append(tokens[index])
        return new_words
    
    def count_tokens(self, tokens):
        unique, counts = np.unique(tokens, return_counts=True)
        return dict(zip(unique, counts))
    
    def retrieve_tokens_times(self, tokens, n):
        tokens = {k: v for k, v in self.count_tokens(tokens).items() if int(v) >= n}
        return tokens 
        
    def preprocessText(self, text):
        processed_text = text
        processed_text = self.remove_tags(processed_text)
        processed_text = self.lower_sentences(processed_text)
        processed_text = self.remove_links(processed_text)
        processed_text = self.remove_punctuation(processed_text)
        tokens = self.segment_word(self.tokenize_sentences(processed_text))
        # print(tokens)
        tokens = self.keep_verb_noun(tokens)
        important_tokens = self.remove_stopwords(tokens, self.stopwords)
        important_tokens = self.remove_digits(important_tokens)
        important_tokens = self.retrieve_tokens_times(important_tokens, 6)
        self.tokens = important_tokens
        return self.tokens

    def __repr__(self):
        return self.text





In [45]:
from sklearn.feature_extraction.text import CountVectorizer
data = []
categories = ['doi-song', 'giai-tri', 'giao-duc', 'khoa-hoc', 'kinh-doanh', 'oto-xe-may', 'suc-khoe', 'the-gioi', 'the-thao']
categories_data = []
# Search all file in folder
for idx, cate in enumerate(categories):
    for root, dirs, files in os.walk(urls.RAW_DATA_PATH+"\\"+cate):
        for file in files:
    ## Read all file in raw_data_test
            file_path = os.path.join(urls.RAW_DATA_PATH+"\\"+cate, file)

            print(file_path)
            exam_str = FileReader(file_path).read()
                

            nlp = NLP(text=exam_str)
            #Segment words
            tokens = nlp.preprocessText(exam_str)
            for key, value in tokens.items():
                data.append(key)
                categories_data.append(idx)
        
            file_names.append(file)
            # topic_str+=(' '.join(tokens))+" "
   
print(categories)
print(data)

# cv=CountVectorizer()
# word_count_vector = cv.fit_transform(train_data)
# print(cv.get_feature_names())
# print(word_count_vector)
# print(word_count_vector.toarray())
# print("\n")        
        




D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10chitiếtnhỏkhiếnphụnữnhìngiàđi.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10câunóicủachamẹcóthểlàmhạitươnglaicon.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10dấuhiệuconbạnđanggặprắcrối.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10hànhvicủachamẹkhiếncontrụctrặckhitrưởngthành.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10hànhvicủatrẻchamẹnênlolắng.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10khácbiệthàihướctrướcvàsaukhicócon.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-song\10kinhnghiệmsốngcủagiáosưĐạihọcHarvard.txt
1943
D:\Learning\Nam4_2\LTMNC\PythonCrawler\newscrawler\predict_category\raw_data\doi-son

In [46]:


print(categories[0])

doi-song


In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

train, test, y_train, y_test = train_test_split(data, categories_data, test_size=0.2, random_state=0)


In [51]:
# create the transform
tfidf_vectorizer = TfidfVectorizer()
tfidf_train = tfidf_vectorizer.fit_transform(train)
tfidf_test = tfidf_vectorizer.transform(test)
# tokenize and build vocab
# tfidf_matrix = vectorizer.fit_transform(train_data)
# # summarize
# print(vectorizer.vocabulary_)
# # print(vectorizer.idf_)
# feature_names = vectorizer.get_feature_names()
# print(feature_names)

# dense = tfidf_matrix.todense()
# denselist = dense.tolist()
# df = pd.DataFrame(denselist, columns=feature_names, index=file_names)
# s = pd.Series(df.loc['filen'])
# s[s > 0].sort_values(ascending=False)[:10]

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)
# # encode document
# vector = vectorizer.transform(["giáo_dục cho học_sinh"])
# # summarize encoded vector
# print(vector.shape)
# print(vector.toarray())

In [55]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(tfidf_train, y_train)
y_pred = nb.predict(tfidf_test)
print('Accuracy test: ',accuracy_score(y_test, y_pred))
print('Accuracy train:',accuracy_score(y_train, nb.predict(tfidf_train)))

Accuracy test:  0.48086734693877553
Accuracy train: 0.7732780612244898


In [86]:
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(categories, y_pred)
print(cm)

NameError: name 'metrics' is not defined

In [85]:
test_example = ["ô_tô xe_máy"]

value_test = tfidf_vectorizer.transform(test_example)
print(categories)
idx = nb.predict(value_test)[0]
print(categories[idx])

['doi-song', 'giai-tri', 'giao-duc', 'khoa-hoc', 'kinh-doanh', 'oto-xe-may', 'suc-khoe', 'the-gioi', 'the-thao']
kinh-doanh
